# Student Performance Classification

## Import

In [1]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner, Evaluator, Pusher
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

## Drop kolom yang tidak dipakai

In [2]:
import pandas as pd

df = pd.read_csv('data/Student_performance_data.csv')

if 'StudentID' in df.columns:
    df.drop(columns=['StudentID'], inplace=True)

    df.to_csv('data/Student_performance_data.csv')

df.head()

,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
0,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
2,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602,4.0
3,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
4,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0


## Set Variable

In [3]:
PIPELINE_NAME = "axelseancp-pipeline"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

In [4]:
DATA_ROOT = "data"

In [5]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Tahapan Data Ingestion

In [6]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="val", hash_buckets=2)
    ])
)

example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [7]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 54
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## Tahapan Data Validation

### Membuat summary statistics

In [8]:
statistics_gen = StatisticsGen(
    examples = example_gen.outputs["examples"]
)
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 55
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [9]:
interactive_context.show(statistics_gen.outputs["statistics"])

### Membuat data schema

In [10]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])

interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 56
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [11]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Absences',INT,required,,-
'Age',INT,required,,-
'Ethnicity',INT,required,,-
'Extracurricular',INT,required,,-
'GPA',FLOAT,required,,-
'Gender',INT,required,,-
'GradeClass',FLOAT,required,,-
'Music',INT,required,,-
'ParentalEducation',INT,required,,-


### Mencari anomali pada dataset

In [12]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs["statistics"],
    schema=schema_gen.outputs["schema"]
)

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 57
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [13]:
interactive_context.show(example_validator.outputs["anomalies"])

## Tahapan Data Preprocessing

In [14]:
TRANSFORM_MODULE_FILE = "student_transform.py"

In [15]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft

def transformed_key(key):
    """Rename transformed key"""
    return key + "_xf"

def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features

    Args:
        inputs: map from feature keys to raw features
    
    Return:
        outputs: map from feature keys to transformed features
    
    Description:
        - apply one hot encoded to categorical features
        - apply standardization to float features and int features that isn't binary
        - apply rename of transformed features except for one hot encoded features
    """

    outputs = {}

    # Standardize features
    outputs[transformed_key("Absences")] = tft.scale_to_0_1(inputs["Absences"])
    outputs[transformed_key("Age")] = tft.scale_to_0_1(inputs["Age"])
    outputs[transformed_key("GPA")] = tft.scale_to_0_1(inputs["GPA"])
    outputs[transformed_key("StudyTimeWeekly")] = tft.scale_to_0_1(inputs["StudyTimeWeekly"])
    outputs[transformed_key("GradeClass")] = tf.cast(inputs["GradeClass"], tf.int64) #Target Feature
    
    # normal features
    outputs["Ethnicity"] = inputs["Ethnicity"]
    outputs["ParentalEducation"] = inputs["ParentalEducation"]
    outputs["ParentalSupport"] = inputs["ParentalSupport"]
    outputs["Extracurricular"] = inputs["Extracurricular"]
    outputs["Gender"] = inputs["Gender"]
    outputs["Music"] = inputs["Music"]
    outputs["Sports"] = inputs["Sports"]
    outputs["Tutoring"] = inputs["Tutoring"]
    outputs["Volunteering"] = inputs["Volunteering"]

    return outputs


Overwriting student_transform.py


In [16]:
transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

INFO:tensorflow:Assets written to: pipelines\axelseancp-pipeline\Transform\transform_graph\58\.temp_path\tftransform_tmp\c68cc6e72031489db5189a425e9dbe1e\assets


INFO:tensorflow:Assets written to: pipelines\axelseancp-pipeline\Transform\transform_graph\58\.temp_path\tftransform_tmp\c68cc6e72031489db5189a425e9dbe1e\assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 58
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## Tahapan Model Training & Tuning

### Membuat Trainer dan Tuner

menggabungkan modul trainer dan tuner

In [17]:
TRAINER_TUNER_MODULE_FILE = "student_trainer_tuner.py"

In [36]:
%%writefile {TRAINER_TUNER_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
import keras_tuner
from keras_tuner.engine import base_tuner
from keras_tuner import RandomSearch
import os
from tfx.components.trainer.fn_args_utils import FnArgs
from typing import NamedTuple, Dict, Text, Any

LABEL_KEY = "GradeClass"

def transformed_key(key):
    """Rename transformed key"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64)->tf.data.Dataset:
    """Get post_transform feature and create batches of data"""

    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )

    # Create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_key(LABEL_KEY)
    )

    return dataset
# BUG
def _get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        # serialized tf example shape [none, 13]

        feature_spec = tf_transform_output.raw_feature_spec()

        feature_spec.pop(LABEL_KEY)

        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        transformed_features = model.tft_layer(parsed_features)

        # get prediction using transformed_features
        return model(transformed_features)
    
    return serve_tf_examples_fn

def get_hyperparameters() -> keras_tuner.HyperParameters:
    """Returns hyperparameters for building model"""
    hp = keras_tuner.HyperParameters()
    hp.Choice('units', [16, 32, 64], default=32)
    hp.Choice('learning_rate', [1e-2, 1e-3], default=1e-3)
    hp.Choice('num_layers', [3, 5, 7], default=3)

    return hp

def model_builder(hparams):
    """Build machine learning model with hyperparameter tuning"""
    
    
    inputs = {
        transformed_key('Age'): tf.keras.layers.Input(shape=[1], name=transformed_key('Age')),
        transformed_key('Absences'): tf.keras.layers.Input(shape=[1], name=transformed_key('Absences')),
        transformed_key('GPA'): tf.keras.layers.Input(shape=[1], name=transformed_key('GPA')),
        transformed_key('StudyTimeWeekly'): tf.keras.layers.Input(shape=[1], name=transformed_key('StudyTimeWeekly')),
        'Ethnicity': tf.keras.layers.Input(shape=[1], name='Ethnicity'),
        'ParentalEducation': tf.keras.layers.Input(shape=[1], name='ParentalEducation'),
        'ParentalSupport': tf.keras.layers.Input(shape=[1], name='ParentalSupport'),
        'Extracurricular': tf.keras.layers.Input(shape=[1], name='Extracurricular'),
        'Gender': tf.keras.layers.Input(shape=[1], name='Gender'),
        'Music': tf.keras.layers.Input(shape=[1], name='Music'),
        'Sports': tf.keras.layers.Input(shape=[1], name='Sports'),
        'Tutoring': tf.keras.layers.Input(shape=[1], name='Tutoring'),
        'Volunteering': tf.keras.layers.Input(shape=[1], name='Volunteering')
    }

    # Concatenate all input features
    concat = layers.Concatenate()(list(inputs.values()))

    x = layers.Dense(hparams.get('units'), activation='relu')(concat)
    for _ in range(int(hparams.get('num_layers'))):
        x = layers.Dense(hparams.get('units'), activation='relu')(x)
    outputs = layers.Dense(5, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.summary()

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hparams.get('learning_rate')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Tuner component will run this function

TunerFnResult = NamedTuple('TunerFnResult', [('tuner', base_tuner.BaseTuner),
                                             ('fit_kwargs', Dict[Text, Any])])

def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
    """
    Build the tuner using the KerasTuner API.
    Args:
        fn_args: Holds args as name/value pairs.
        - working_dir: working dir for tuning.
        - train_files: List of file paths containing training tf.Example data.
        - eval_files: List of file paths containing eval tf.Example data.
        - train_steps: number of train steps.
        - eval_steps: number of eval steps.
        - schema_path: optional schema of the input data.
        - transform_graph_path: optional transform graph produced by TFT.
    Returns:
        A namedtuple contains the following:
        - tuner: A BaseTuner that will be used for tuning.
        - fit_kwargs: Args to pass to tuner's run_trial function for fitting the
                        model , e.g., the training and validation dataset. Required
                        args depend on the above tuner's implementation.
    """

    # Define tuner
    tuner = RandomSearch(
        model_builder,
        objective='val_accuracy',
        max_trials=10,
        hyperparameters=get_hyperparameters(),
        directory=fn_args.working_dir,
        project_name='student_performance_classification'
    )

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)

    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            'x': train_set,
            'validation_data': val_set
        }
    )

# Trainer component will run this function
def run_fn(fn_args: FnArgs) -> None:
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )

    es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, verbose=1)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)

    hparams = keras_tuner.HyperParameters.from_config(fn_args.hyperparameters)

    # Build the model with tuned hyperparameters
    model = model_builder(hparams)

    # Train the model
    model.fit(
        x = train_set,
        validation_data = val_set,
        callbacks = [tensorboard_callback, es, mc],
        epochs=10,
        verbose=2
    )

    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
            tf.TensorSpec(
                shape=[None],
                dtype=tf.string,
                name='examples'
            )
        )
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting student_trainer_tuner.py


In [37]:
from tfx.proto import trainer_pb2

### Membuat komponen Tuner

In [38]:
tuner = Tuner(
    module_file = os.path.abspath(TRAINER_TUNER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema = schema_gen.outputs['schema'],
    train_args = trainer_pb2.TrainArgs(splits=['train']),
    eval_args = trainer_pb2.EvalArgs(splits=['val'])
)

interactive_context.run(tuner)

Trial 10 Complete [00h 00m 01s]
val_accuracy: 0.7757575511932373

Best val_accuracy So Far: 0.8848484754562378
Total elapsed time: 00h 00m 17s
Results summary
Results in pipelines\axelseancp-pipeline\.temp\68\student_performance_classification
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 00 summary
Hyperparameters:
units: 16
learning_rate: 0.01
num_layers: 5
Score: 0.8848484754562378

Trial 03 summary
Hyperparameters:
units: 32
learning_rate: 0.01
num_layers: 5
Score: 0.8828282952308655

Trial 08 summary
Hyperparameters:
units: 16
learning_rate: 0.01
num_layers: 3
Score: 0.8767676949501038

Trial 07 summary
Hyperparameters:
units: 64
learning_rate: 0.001
num_layers: 7
Score: 0.8727272748947144

Trial 05 summary
Hyperparameters:
units: 32
learning_rate: 0.001
num_layers: 7
Score: 0.8484848737716675

Trial 06 summary
Hyperparameters:
units: 64
learning_rate: 0.001
num_layers: 3
Score: 0.8484848737716675

Trial 01 summary
Hyperparameters:
units: 32
learnin

ExecutionResult(
    component_id: Tuner
    execution_id: 68
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### Membuat komponen Trainer

In [39]:
trainer = Trainer(
    module_file = os.path.abspath(TRAINER_TUNER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema = schema_gen.outputs['schema'],
    hyperparameters = tuner.outputs['best_hyperparameters'],
    train_args = trainer_pb2.TrainArgs(splits=['train']),
    eval_args = trainer_pb2.EvalArgs(splits=['val'])
)

interactive_context.run(trainer)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Age_xf (InputLayer)         [(None, 1)]                  0         []                            
                                                                                                  
 Absences_xf (InputLayer)    [(None, 1)]                  0         []                            
                                                                                                  
 GPA_xf (InputLayer)         [(None, 1)]                  0         []                            
                                                                                                  
 StudyTimeWeekly_xf (InputL  [(None, 1)]                  0         []                            
 ayer)                                                                                      

INFO:tensorflow:Assets written to: pipelines\axelseancp-pipeline\Trainer\model\69\Format-Serving\assets


297/297 - 2s - loss: 0.6698 - accuracy: 0.7888 - val_loss: 0.5169 - val_accuracy: 0.8343 - 2s/epoch - 6ms/step
Epoch 2/10

Epoch 2: val_accuracy improved from 0.83434 to 0.83636, saving model to pipelines\axelseancp-pipeline\Trainer\model\69\Format-Serving
INFO:tensorflow:Assets written to: pipelines\axelseancp-pipeline\Trainer\model\69\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\axelseancp-pipeline\Trainer\model\69\Format-Serving\assets


297/297 - 1s - loss: 0.4584 - accuracy: 0.8815 - val_loss: 0.5908 - val_accuracy: 0.8364 - 1s/epoch - 3ms/step
Epoch 3/10

Epoch 3: val_accuracy improved from 0.83636 to 0.87879, saving model to pipelines\axelseancp-pipeline\Trainer\model\69\Format-Serving
INFO:tensorflow:Assets written to: pipelines\axelseancp-pipeline\Trainer\model\69\Format-Serving\assets


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\axelseancp-pipeline\Trainer\model\69\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\axelseancp-pipeline\Trainer\model\69\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 69
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)